## 0. Description

#### INPUT DATASET: 
#####                    (1)	PDS records of 10 sample districts - Citizen_Endline_Final_Sample_Unique.csv
##### 					 (2)	Wards in our sample
#####                    (3)    Voter roll files from 10 sample districts
     
#### OUTPUT DATASETS: 	
#####                    (1)    Sample for phone survey

#### DESCRIPTION: 		
#####                    We fuzzy merge citizen records from Public Distribution System (PDS) and electoral voter roll
#####                    to generate a sample for the phone survey

In [ ]:
# !pip install nbconvert

In [ ]:
# !pip install pandoc

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import csv
import html5lib
from pathlib import Path
import glob

In [ ]:
import re
import random
import time

In [ ]:
import os

import unicodedata

from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

In [ ]:
import random

In [ ]:
from fuzzywuzzy import fuzz, process
# from rapidfuzz import fuzz, process

In [ ]:
#Set up paths
db = '/Users/swathi/Library/CloudStorage/Dropbox/Peer Effects and Role Models/Analysis_Experiment/'
voter_roll = '1_Raw/Input/Citizen Endline/Voter_Roll/Converted/csvs_3/'
pds_input = '1_Raw/Input/Citizen Endline/PDS/'
sample_input = '3_Analyse/Input/endline_citizen/'

## 1. Open our WM sample

In [ ]:
#To get the UGP, open our sample dataset as a dataframe
wm_sample_path = '1_Raw/Input/Collective Action Experiment/treated_and_enrolled_wards.dta'
wm_sample_file = os.path.join(db, wm_sample_path)
wm_df = pd.read_stata(wm_sample_file)
# wm_df

In [ ]:
wm_df['SpilloverWard'].unique()

In [ ]:
print(f"Unique wards in treat+control+spill: {wm_df.shape[0]}")

In [ ]:
wm_ugp = wm_df[['DistrictName', 'SubDistrictName', 'GramPanchayatName', 'WardNo', 'UGP']]

In [ ]:
wm_ugp['WardNo'].unique()

In [ ]:
type(wm_ugp['WardNo'][0])

In [ ]:
wm_ugp.loc[:, 'WardNo'] = wm_ugp.loc[:, 'WardNo'].astype('int')

In [ ]:
wm_ugp.shape

## 1.1 Open PDS sample

In [ ]:
pds_file_name = 'Citizen_Endline_Final_Sample_Unique.csv'

pds_file = os.path.join(db, sample_input, pds_file_name)
pds_df = pd.read_csv(pds_file, dtype={'Mobile':'str'}, low_memory=False)

## 2. Open all voter roll .csv files downloaded

In [ ]:
voter_csv_path = os.path.join(db, voter_roll)

In [ ]:
files_list = [file for file in os.listdir(voter_csv_path) if file.endswith('.csv') and 'Copy' not in file]
print(files_list)

In [ ]:
# Find which all of the files are actually in our sample
voter_files = []
for file in files_list:
    dist = file.split('_')[0] if len(file.split('_'))>0 else ''
    block = file.split('_')[1] if len(file.split('_'))>1 else ''
    gp = file.split('_')[2] if len(file.split('_'))>2 else ''
    ward = file.split('_')[3] if len(file.split('_'))>3 else ''
    
    ward = ward.split('.')[0] if len(ward.split('.'))>0 else ''
    # geo_id = file.split('_')
    print(dist, block, gp, ward)
    file_dict = {'District':dist, 'Block':block, 'GramPanchayat':gp, 'WardNo':ward}
    # file_dict = {geo_id}
    voter_files.append(file_dict)
print(voter_files)    

In [ ]:
voter_files = pd.DataFrame(voter_files)

In [ ]:
voter_files.loc[voter_files['WardNo']=='']

In [ ]:
voter_files.loc[voter_files['GramPanchayat']=='']

In [ ]:
voter_files.loc[voter_files['Block']=='']

In [ ]:
voter_files.loc[voter_files['District']=='']

In [ ]:
voter_files = voter_files.loc[voter_files['WardNo']!='']

In [ ]:
print(f"Unique wards downloaded: {pd.concat([voter_files['District'], voter_files['Block'], voter_files['GramPanchayat'], voter_files['WardNo']]).nunique()}")

In [ ]:
voter_files = voter_files.rename(columns={'District': 'DistrictName', 'Block':'SubDistrictName', 'GramPanchayat':'GramPanchayatName'})

In [ ]:
voter_files['WardNo'] = voter_files['WardNo'].astype('int')

In [ ]:
voter_files.columns

## 3. Find out which all of the csv files downloaded are actually in our sample

### 3.1 Fuzzy merge voter roll and sample districts

In [ ]:
#Fuzzy merge voter files and ward member sample
for voter_dist in voter_files['DistrictName'].unique():
    wm_dist = (wm_ugp['DistrictName'].unique()).tolist()
    voter_match_dist = process.extractOne(voter_dist, wm_dist)
    print(voter_dist, voter_match_dist)
    voter_files.loc[voter_files['DistrictName']==voter_dist, 'Matched_District']=voter_match_dist[0]
    print(voter_files['Matched_District'])

In [ ]:
voter_files[['DistrictName', 'Matched_District']].drop_duplicates()

### 3.2 Fuzzy merge voter roll and sample blocks

In [ ]:
#Fuzzy match block
for voter_dist in voter_files['Matched_District'].unique():
    wm_blocks = (wm_ugp[wm_ugp['DistrictName']==voter_dist]['SubDistrictName'].unique()).tolist()
    for voter_block in voter_files[voter_files['Matched_District']==voter_dist]['SubDistrictName'].unique():
        # print(voter_dist, voter_block)
        # print(len(wm_blocks))
        # voter_block = voter_block.upper()
        voter_match_block = process.extractOne(voter_block.upper(), wm_blocks)
        # print(voter_block, voter_match_block)
        voter_files.loc[(voter_files['Matched_District']==voter_dist) & (voter_files['SubDistrictName']==voter_block), 'Matched_Block'] = voter_match_block[0]
        voter_files.loc[(voter_files['Matched_District']==voter_dist) & (voter_files['SubDistrictName']==voter_block), 'Block_match_score'] = voter_match_block[1]
        # break
    # break

In [ ]:
wm_ugp['DistrictName'].unique()

In [ ]:
voter_files[['DistrictName', 'SubDistrictName', 'Matched_Block']].drop_duplicates()[150:]

### 3.3 Fuzzy merge voter roll and sample GPs

In [ ]:
#Fuzzy match GP
for voter_dist in voter_files['Matched_District'].unique():
    for voter_block in voter_files[voter_files['Matched_District']==voter_dist]['Matched_Block'].unique():
        wm_gp_list = (wm_ugp[(wm_ugp['DistrictName']==voter_dist) & (wm_ugp['SubDistrictName']==voter_block)]['GramPanchayatName'].unique()).tolist()
        # print(voter_dist, voter_block, wm_gp_list)
        for voter_gp in voter_files[(voter_files['Matched_District']==voter_dist) & (voter_files['Matched_Block']==voter_block)]['GramPanchayatName'].unique():
            # print(voter_dist, voter_block, voter_gp)
            voter_match_gp = process.extractOne(voter_gp.upper(), wm_gp_list)
            print(voter_gp, voter_match_gp)            
            voter_files.loc[(voter_files['Matched_District']==voter_dist) & (voter_files['Matched_Block']==voter_block) & (voter_files['GramPanchayatName']==voter_gp), 'Matched_GP'] = voter_match_gp[0]
            voter_files.loc[(voter_files['Matched_District']==voter_dist) & (voter_files['Matched_Block']==voter_block) & (voter_files['GramPanchayatName']==voter_gp), 'GP_match_score'] = voter_match_gp[1]
            # break
        # # break

In [ ]:
voter_files['GP_match_score'].describe()

In [ ]:
voter_gp_check = (voter_files.loc[(voter_files['GP_match_score']<80)][['Matched_District', 'Matched_Block', 'GramPanchayatName', 'Matched_GP', 'GP_match_score']])

In [ ]:
voter_gp_check.drop_duplicates()[30:]

In [ ]:
# voter_files.loc[(voter_files['Matched_District']=='PURNIA') & (voter_files['Matched_Block']=='AMOUR') & (voter_files['GramPanchayatName']=='Jhouwari'), 'Matched_GP']=np.nan

In [ ]:
voter_files.loc[voter_files.duplicated(subset=['Matched_District', 'Matched_Block', 'GramPanchayatName', 'WardNo'], keep=False)]

In [ ]:
# (voter_files.loc[(voter_files['Matched_District']=='PURBI CHAMPARAN') & (voter_files['Matched_Block']=='TURKAULIA')]['GramPanchayatName']).unique()

In [ ]:
voter_files.loc[(voter_files['Matched_Block'].isna()) | (voter_files['Matched_GP'].isna())]

In [ ]:
#Keep only the ones where Matched_Block and Matched_SubDistrict are no-missing and map ugp to them
voter_sample_files = voter_files.loc[(voter_files['Matched_Block'].notna()) & (voter_files['Matched_GP'].notna())]

In [ ]:
voter_sample_files['WardNo'].dtype

In [ ]:
wm_ugp['WardNo'].dtype

In [ ]:
wm_ugp.loc[:, 'WardNo'] = wm_ugp.loc[:, 'WardNo'].astype('int')

In [ ]:
voter_sample_files.loc[:, 'WardNo'] = voter_sample_files.loc[:, 'WardNo'].astype('int')

In [ ]:
voter_files.shape

In [ ]:
voter_sample_files.shape

In [ ]:
voter_sample_files = voter_sample_files.rename(columns={'WardNo':'WardNo_Voter'})

### 3.4 Merge voter roll and sample files on district_block_gp and wardnumber

In [ ]:
voter_sample_files = pd.merge(voter_sample_files, wm_ugp, left_on=['Matched_District', 'Matched_Block', 'Matched_GP', 'WardNo_Voter'], right_on=['DistrictName', 'SubDistrictName', 'GramPanchayatName', 'WardNo'], how='left', suffixes=('', '_wm'))

In [ ]:
voter_sample_files.WardNo.dtype

In [ ]:
voter_sample_files.WardNo_Voter.dtype

In [ ]:
voter_sample_files.shape

In [ ]:
voter_sample_files.loc[voter_sample_files['UGP'].notnull()]

In [ ]:
voter_sample_files.WardNo_Voter.nunique()

In [ ]:
voter_sample_files['WardNo'].nunique()

In [ ]:
voter_sample_files.loc[(voter_sample_files['WardNo'].notnull())]['WardNo_Voter'].nunique()

In [ ]:
voter_sample_files = voter_sample_files.loc[(voter_sample_files['UGP'].notnull()) & (voter_sample_files['WardNo'].notnull())]

In [ ]:
voter_sample_files = voter_sample_files[['DistrictName', 'SubDistrictName', 'GramPanchayatName', 'WardNo','Matched_District', 'Matched_Block', 'Block_match_score', 'Matched_GP','GP_match_score', 'UGP']]

In [ ]:
voter_sample_files.shape

In [ ]:
voter_sample_files.columns

In [ ]:
print(f"No.of unique UGPs in voter files from our WM sample: {voter_sample_files.shape[0]}")

In [ ]:
voter_files.shape

In [ ]:
voter_sample_files['WardNo'] = voter_sample_files['WardNo'].astype('int')

In [ ]:
voter_sample_files

### 3.5 Add UGP identifier to all voter roll csv files in our sample

In [ ]:
voter_sample_files['WardNo'] = voter_sample_files['WardNo'].astype('str')

In [ ]:
voter_sample_files['filename'] = voter_sample_files['DistrictName'] + "_" + voter_sample_files['SubDistrictName'] + "_" + voter_sample_files['GramPanchayatName'] + "_" + voter_sample_files['WardNo'] + ".csv"

In [ ]:
voter_sample_files['WardNo'] = voter_sample_files['WardNo'].astype('int')

In [ ]:
# voter_sample_files.loc[voter_sample_files['filename']=='BANKA_Rajoun_Orhara_10.csv']

In [ ]:
voter_sample_files.filename.unique()

In [ ]:
all_voter_files_num = 0
voter_all_path = os.path.join(db, voter_roll)
for file in os.listdir(voter_all_path):
    # print(file)
    if file in voter_sample_files['filename'].unique():
        print(file)
        all_voter_files_num +=1
        voter_file_csv = pd.read_csv(os.path.join(voter_all_path, file))
        ugp = list(voter_sample_files[voter_sample_files['filename']==file]['UGP'])
        district = list(voter_sample_files[voter_sample_files['filename']==file]['Matched_District'])
        block = list(voter_sample_files[voter_sample_files['filename']==file]['Matched_Block'])
        gp = list(voter_sample_files[voter_sample_files['filename']==file]['Matched_GP'])
        wardno = list(voter_sample_files[voter_sample_files['filename']==file]['WardNo'])
        # print(voter_file_csv['ward_pdf'].dtype)
        # if wardno[0] not in voter_file_csv['ward_pdf']:
        #     print(file)
        #     print(wardno)
        #     print(voter_file_csv['ward_pdf'])
        #     break
        try:
            voter_file_csv['UGP'] = ugp[0]
            voter_file_csv['DistrictName'] = district[0]
            voter_file_csv['SubDistrictName'] = block[0]
            voter_file_csv['GramPanchayatName'] = gp[0]
            # print(f'{type(ugp)}')
            # print(ugp)
            # print(voter_file_csv)
            # break
        except Exception as e:
            print(f'UGP not found for file {file}: {e}')
            print(f'{type(ugp)}')
            print(ugp[0])
            break

        #Now, write it back to csv
        try:
            voter_file_csv.to_csv(os.path.join(voter_all_path, file), index=False)
            print(f'{file} updated')
            # break
        except Exception as e:
            print(f'{file} not updated: {e}')
            break

#         # #Read back the first one
#         # # voter_file_csv = pd.read_csv(os.path.join(voter_all_path, file))
#         # # print(voter_file_csv)
#         # # break
print(all_voter_files_num)

In [ ]:
voter_file_csv.columns

## 4. Move all voter roll csv files in our sample to a separate folder

In [ ]:
file_nums = []
for file in os.listdir(os.path.join(db, voter_roll)):
    if os.path.isdir(os.path.join(db, voter_roll, file)):
        try:
            # print(file)
            sam_num = int(file.split('_')[-1])
            # print(sam_num)
            file_nums.append(sam_num)
        except ValueError:
            print(f"{file} does not end with a number.")
            continue
# print(file_nums)
last_sample = max(file_nums)
print(last_sample)
voter_sample_folder = f"Files_in_Sample_SET_{last_sample+1}"
print(voter_sample_folder)

In [ ]:
#Now, move all the csv files in voter_df_merged - which are the ones in our sample to a different folder
if not os.path.exists(os.path.join(db, voter_roll, voter_sample_folder)):
    os.makedirs(os.path.join(db, voter_roll, voter_sample_folder))
print(os.path.join(db, voter_roll, voter_sample_folder))

In [ ]:
voter_all_path = os.path.join(db, voter_roll)
for file in os.listdir(voter_all_path):
    print(file)

In [ ]:
voter_sample_path = os.path.join(db, voter_roll, voter_sample_folder)
voter_all_path = os.path.join(db, voter_roll)
for file in os.listdir(voter_all_path):
    if file in voter_sample_files['filename'].unique():
        try:
            os.rename(os.path.join(voter_all_path, file), os.path.join(voter_sample_path, file))
        except Exception as e:
            print(f"Exception for {file}: {e}")

In [ ]:
#No. of files in each path
voter_all_path = os.path.join(db, voter_roll)
sample_files_list = []
for file in os.listdir(voter_sample_path):
    if file.endswith('.csv'):
        sample_files_list.append(file)
print(f"No.of voter roll csvs in sample: {len(sample_files_list)}")

all_files_list = []
for file in os.listdir(voter_all_path):
    if file!='.DS_Store' and file!='all_files.csv' and file.endswith('.csv'):
        all_files_list.append(file)
print(f"No.of voter roll csvs not in sample: {len(all_files_list)}")

### 4.1 Out of those in our sample, isolate those where ward_pds does not match with the ward number in the name of the file

In [ ]:
unmatch_csvs = []
for file in os.listdir(voter_sample_path):
    if file.endswith('.csv'):
        # print(file)
        df = pd.read_csv(os.path.join(voter_sample_path, file))
        # print(df.columns)
        ward_pdf = df['ward_pdf'][0]
        # print(ward_pdf)
        filename_ward = file.split('_')[-1].split('.')[0]
        # print(filename_ward)
        if ward_pdf==np.nan:
            print(f"Ward inside csv file: {ward_pdf}")
            print(f"Ward in file name: {filename_ward}")
            unmatch_csvs.append(file)
            continue
        try:
            if float(ward_pdf)!=float(filename_ward):
                # print(file)
                # print(f"Ward inside csv file: {ward_pdf}")
                # print(f"Ward in file name: {filename_ward}")
                unmatch_csvs.append(file)
                # break
        except ValueError:
            print(f"Value error for {file}. Ward PDF: {ward_pdf}")
            unmatch_csvs.append(file)

len(unmatch_csvs)

In [ ]:
### Open the csvs where ward_pdf is different from ward in filename, replace ward_pdf with ward in filename, overwrite the csvs and move them to a new folder

for file in os.listdir(voter_sample_path):
    if file in unmatch_csvs:
        print(file)
        df = pd.read_csv(os.path.join(voter_sample_path, file))
        filename_ward = file.split('_')[-1].split('.')[0]
        if len(filename_ward)!=0:
            # print(df['ward_pdf'][0])
            df.loc[:, 'ward_pdf'] = float(filename_ward)
            # print(df['ward_pdf'][0])
            df.to_csv(os.path.join(voter_sample_path, file), index=False)

### 4.1 Open all voter roll csv file and add a random order column at individual level if they don't already have one

In [ ]:
for file in os.listdir(voter_sample_path):
    voter_file = pd.read_csv(os.path.join(voter_sample_path, file))
    voter_file_cols = list(voter_file.columns)
    # print(voter_file_cols)
    if 'Voter_Random_Order' not in voter_file_cols:
        voter_file['Voter_Random_Order'] = [random.randint(1, 100000000) for k in voter_file.index]
        # print(voter_file['Voter_Random_Order'])
        voter_file.to_csv(os.path.join(voter_sample_path, file), index=False)
        # break
    else:
        print(f'Random number there already for file {file}')
        # break

### 4.2 Give each voter roll csv file a random number ordering

In [ ]:
start = time.perf_counter()
for file in os.listdir(voter_sample_path):
    print(file)
    file_path = os.path.join(voter_sample_path, file)
    # print(file_path)
    voter_ward_id = random.randint(1, 10000)
    newfile = str(voter_ward_id) + '_' + file
    newfile_path = os.path.join(voter_sample_path, newfile)
    # print(newfile_path)
    try:
        os.rename(file_path, newfile_path)
        print(f"{file} renamed to {newfile}.")
        # break
    except IsADirectoryError:
        print(f"{file} is a directory.")
        break
end = time.perf_counter()
time_taken = end-start
print(f"Time taken = {time_taken:.3f} seconds.")

In [ ]:
# voter_sample_folder = 'Files_in_Sample_SET_31'
# voter_sample_path = os.path.join(db, voter_roll, voter_sample_folder)

In [ ]:
#Now sort the files by their random order
sample_files = glob.iglob(voter_sample_path + "/*.csv")
sample_files_sorted = sorted(sample_files, key = lambda x: int(x.split('/')[-1].split('_')[0]))
# for file in sample_files_sorted:
#     print(file)

In [ ]:
# #Check if Files_in_Sample_SET3 has duplicates
# # sample_files = glob.glob(voter_sample_path + "/*.csv")
# sample_file_dict = {}
# for file in os.listdir(voter_sample_path):
#     sample_file_dict[file] = 0
#     # print(sample_file_dict[file])
# for file in os.listdir(voter_sample_path):
#     sample_file_dict[file] = int(sample_file_dict[file])+1
#     # print(sample_file_dict[file])
#     # break

## 5. Open all voter roll csv files in our sample and append to a dataframe

In [ ]:
## 5. Open all voter roll csv files in our sample and append to a dataframe
voter_sample_files = []
for file in sample_files_sorted:
    # print(file)
    # voter_file = pd.read_csv(os.path.join(voter_sample_path, file))
    voter_file = pd.read_csv(file)
    # print(type(voter_file))
    voter_sample_files.append(voter_file)
voter_df = pd.concat(voter_sample_files)

In [ ]:
print(f'Total observations in sampled voter csv files before de-duplicating: {voter_df.shape[0]}')

In [ ]:
voter_df.columns

In [ ]:
#Check for duplicates
voter_df[voter_df.duplicated(subset=['UGP', 'ward_pdf', 'elector_name', 'father_or_husband_name'], keep='first')].sort_values(['UGP', 'ward_pdf', 'elector_name', 'father_or_husband_name'])[['UGP', 'ward_pdf', 'elector_name', 'father_or_husband_name']]

In [ ]:
voter_df = voter_df.drop_duplicates(subset=['UGP', 'ward_pdf', 'elector_name', 'father_or_husband_name'], keep='first').sort_values(['UGP', 'ward_pdf', 'elector_name', 'father_or_husband_name'])

In [ ]:
print(f"No.of observations in sample voter csvs after de-duplicating: {voter_df.shape[0]}")

## 6. Merge with full PDS sample and create a dataframe with only the wards for which we have voter roll csvs

In [ ]:
type(pds_df['Mobile'][0])

In [ ]:
pds_df.shape

In [ ]:
pds_df.columns

In [ ]:
print(f"No.of observations in PDS full sample: {pds_df.shape[0]}")

In [ ]:
pds_df['UGP'] = pds_df['UGP'].astype('float')

In [ ]:
pds_df['UGP'].nunique()

In [ ]:
### 6.1 Removing duplicates and invalid phone numbers from full PDS sample and saving a csv -->

In [ ]:
# pds_df['Mobile'] = pds_df['Mobile'].map(
#     lambda x: re.sub("[^0-9]", "", x))

In [ ]:
# pds_df = pds_df.loc[pds_df['Mobile'].str.len()==10]

In [ ]:
# pds_df = pds_df.drop_duplicates(subset=['Mobile'], keep='first')

In [ ]:
# pds_df.shape[0]

In [ ]:
# pds_df.to_csv(os.path.join(db, pds_input, 'Citizen Endline Sample Priority F.csv'), index=False)

### 6.2 Create dataframe with only UGP and ward in sample voter roll files

In [ ]:
pds_df.shape

In [ ]:
# Create dataframe with only UGP and ward in sample voter roll files
pds_df[pds_df['UGP'].isin(voter_df['UGP'].unique())]['UGP'].nunique()

In [ ]:
voter_df['UGP'].nunique()

In [ ]:
# print('Voter files have 836 unique UGP but PDS sample has only 831')

In [ ]:
pds_df_slice = pds_df[pds_df['UGP'].isin(voter_df['UGP'].unique())]

In [ ]:
pds_df_slice.columns

In [ ]:
print(f"No.of observations in PDS sample with UGPs in our sample: {pds_df_slice.shape[0]}")

In [ ]:
pds_allnames = pds_df_slice['AllNames'].str.split(',', expand=True)
# print(type(pds_allnames))
pds_allnames.columns = [f"Member_{i+1}" for i in range(pds_allnames.shape[1])]
pds_df_slice = pd.concat([pds_df_slice, pds_allnames], axis=1)

### 6.3 Transliterate

In [ ]:
pds_df_slice.columns

In [ ]:
#Put all member columns to a list
columns_to_trans = []
for col in pds_df_slice.columns:
    if ('Member' in col or col=='Father_Name') and col!='Member_ID':
        # print(col)
        columns_to_trans.append(col)
print(columns_to_trans)

In [ ]:
#Transliterate the Hindi names into English

pds_df_slice[columns_to_trans] = pds_df_slice[columns_to_trans].map(
    lambda x: unicodedata.normalize('NFC', x) if isinstance(x, str) else x
)

pds_df_slice[columns_to_trans] = pds_df_slice[columns_to_trans].map(
    lambda x: transliterate(x, sanscript.DEVANAGARI, sanscript.ITRANS) if isinstance(x, str) else x
)

pds_df_slice[columns_to_trans] = pds_df_slice[columns_to_trans].map(
    lambda x: x.upper() if isinstance(x, str) else x
)

In [ ]:
voter_cols_to_translate = ['elector_name', 'father_or_husband_name']

voter_df[voter_cols_to_translate] = voter_df[voter_cols_to_translate].map(
    lambda x: unicodedata.normalize('NFC', x) if isinstance(x, str) else x
)

voter_df[voter_cols_to_translate] = voter_df[voter_cols_to_translate].map(
    lambda x: transliterate(x, sanscript.DEVANAGARI, sanscript.ITRANS) if isinstance(x, str) else x
)

voter_df[voter_cols_to_translate] = voter_df[voter_cols_to_translate].map(
    lambda x: x.upper() if isinstance(x, str) else x
)

In [ ]:
voter_df.columns

In [ ]:
voter_df = voter_df.rename(columns={'ward_pdf':'WardNo'})

In [ ]:
pds_df_slice.columns

In [ ]:
pds_col_names = list(pds_df_slice.columns.values)

In [ ]:
for col in pds_col_names:
    print(pds_col_names.index(col), col)

In [ ]:
#Convert all '' and None to null
pds_df_slice = pds_df_slice.iloc[:, :].replace('', np.nan)

In [ ]:
for col in pds_df_slice.columns:
    pds_df_slice.loc[pds_df_slice[col]=='', col] = np.nan

In [ ]:
voter_df.columns

In [ ]:
voter_df.loc[(voter_df['father_or_husband_name']==''), 'father_or_husband_name'] = np.nan

In [ ]:
voter_df.loc[(voter_df['elector_name']==''), 'elector_name'] = np.nan

In [ ]:
print(f"In voter df, {voter_df.loc[(voter_df['father_or_husband_name'].isnull())|(voter_df['elector_name'].isnull())].shape[0]} observations have either elector or guardian name missing.")

In [ ]:
#Remove where elector or guardian is missing
voter_df = voter_df[(voter_df['father_or_husband_name'].notnull()) & (voter_df['elector_name'].notnull())]

In [ ]:
print(f'After removing rows where either voter or guardian is missing, we have {voter_df.shape[0]} rows.')

In [ ]:
voter_df.reset_index(inplace=True)

## 6.4 Sort voter and PDS by random order

In [ ]:
#Sort pds by random order
pds_df_slice = pds_df_slice.sort_values(by=['UGP', 'RandomOrder'])

In [ ]:
#Sort voter roll by random order
voter_df = voter_df.sort_values(by=['UGP', 'WardNo', 'Voter_Random_Order'])

In [ ]:
voter_df['Voter_Random_Order'].nunique()

In [ ]:
voter_df.shape[0]

## 7. Do the fuzzy matching

In [ ]:
# #Now, do the fuzzy matching
def fuzzy_match(voter, members, threshold=90):
    # voter = voter if pd.notnull(voter) else ""
    # for member in members:
        # member = member if pd.notnull(member) else ""
        # score = fuzz.token_sort_ratio(voter, member)
    matched_voter_guard, score = process.extractOne(voter, members, scorer=fuzz.token_set_ratio)
    # print(process.extractOne(voter, members, scorer=fuzz.token_sort_ratio))
    if score >=threshold:
        # print(f'Voter/Guardian: {voter}, All members: {members}, Matched member:{member}, Score:{score}')
        # print(f'Voter/Guardian: {voter}, Matched member:{matched_voter_guard}, Score:{score}')
        return score, matched_voter_guard
    else:
        return 0, 'NaN'

In [ ]:
# ugp=497
# print(voter_df_merged.shape)
# voter_df_merged_notnull = voter_df_merged[(voter_df_merged['elector_name'].notnull()) & (voter_df_merged['father_or_husband_name'].notnull())]
# voter_df_merged_notnull[voter_df_merged_notnull['UGP']==ugp]

In [ ]:
print(f"Voter: {voter_df.shape[0]}, PDS: {pds_df_slice.shape[0]}")

In [ ]:
voter_df = voter_df.rename(columns={'UGP':'UGP_Voter'})

In [ ]:
np.where(voter_df['UGP_Voter'].unique()==6816)

In [ ]:
voter_sample_folder

In [ ]:
# voter_df['UGP_Voter'].unique().index

In [ ]:
voter_df['UGP_Voter'].nunique()

In [ ]:
# ugp_list = []
# for ugp in voter_df['UGP_Voter'].unique():
#     if ugp not in ugp_list:
#         print(f"UGP: {ugp}; Index: {np.where(voter_df['UGP_Voter'].unique()==ugp)}")
#         ugp_list.append(ugp)

In [ ]:
%%time
matched_rows = []
for ugp in voter_df['UGP_Voter'].unique():
    for ward in voter_df[voter_df['UGP_Voter']==ugp]['WardNo'].unique():
        match_count = 0
        for voter_idx, voter_row in voter_df[(voter_df['UGP_Voter']==ugp) & (voter_df['WardNo']==ward)].iterrows():
            for pds_idx, pds_row in pds_df_slice.loc[(pds_df_slice['UGP']==ugp)].iterrows():
                try:
                    member_list = pds_row[columns_to_trans]
                    # print(member_list)
                    member_list = list(member_list[member_list.notnull()])
                    # print(len(member_list))
                    # print(f'Voter:{voter_row['elector_name']}')
                    if len(member_list)!=0:
                        voter_match, matched_voter_guard = fuzzy_match(voter_row['elector_name'], member_list)
                        # print(voter_match, matched_voter_guard)
                        # print(f'Guard:{voter_row['father_or_husband_name']}')
                        guard_match, guard_match_mem = fuzzy_match(voter_row['father_or_husband_name'], member_list)
                        # print(guard_match, guard_match_mem)
                    else:
                        voter_match, matched_voter_guard = (0, 'NaN')
                        guard_match, guard_match_mem = (0, 'NaN')
                        # print(voter_match, matched_voter_guard)
                        # print(guard_match, guard_match_mem)
                        # break
                except Exception as e:
                    print(f"Exception for file {voter_row['filename']} and mobile {pds_row['Mobile']} with members {member_list},ii voter_name {voter_row['elector_name']}, guardian_name {voter_row['father_or_husband_name']}")
                    break
                if voter_match>=90 and guard_match>=90:
                    total_match_score = voter_match + guard_match
                    # print(f"UGP:{ugp}, Ward:{ward}, Voter ID:{voter_idx}, PDS ID: {pds_idx}, Matches reached: {match_count}")
                    # print(f"Voter:{voter_row['elector_name']}, Voter match mem: {matched_voter_guard}, Voter_match_score:{voter_match}, Guard:{voter_row['father_or_husband_name']}, Guard match mem: {guard_match_mem}, Guard match score:{guard_match}")
                    # if voter_match<=85 or guard_match<=85:
                    #     print(f"Voter:{voter_row['elector_name']}, Voter match mem: {voter_match_mem}, Voter_match_score:{voter_match}, Guard:{voter_row['father_or_husband_name']}, Guard match mem: {guard_match_mem}, Guard match score:{guard_match}")
                    matched_row = pd.concat([pds_row, voter_row], axis=0)
                    matched_row['Voter_Match_Score'] = voter_match
                    matched_row['Guardian_Match_Score'] = guard_match
                    matched_row['Total_Match_Score'] = total_match_score
                    matched_rows.append(matched_row)
                    # pds_df_slice = pds_df_slice.drop(axis=0, index=pds_idx)
                    match_count += 1
                    break
                # break
            # break
            if match_count==25:
                for el in matched_rows:
                    el['Voter_Stop_Random_Number'] = voter_row['Voter_Random_Order']
                break
        print(f"UGP: {ugp}, Ward: {ward}, Matches reached: {match_count}")
        # break
    # break
matched_rows = pd.DataFrame(matched_rows)
    # break

In [ ]:
#When took 90 as the cutoff, got 13 matches in a ward (got 16 with 85, but all below 90 were wrong)
#After token_set_ratio, 15 above 90, 38, 54
#Took 3 hours for 40 UGPs

In [ ]:
# matched_rows = pd.DataFrame(matched_rows)

In [ ]:
# matched_rows = matched_rows.loc[matched_rows['UGP_Voter']!=5572.00]

In [ ]:
# np.where(matched_rows['UGP_Voter']==5572)

In [ ]:
#If same PDS mobile matched with multiple voters, keep one with higher score
matched_rows[matched_rows.duplicated(subset=['Mobile'], keep='first')].sort_values(by=['Mobile', 'Total_Match_Score'])

In [ ]:
matched_rows = matched_rows.sort_values(by=['Mobile', 'Total_Match_Score'])

In [ ]:
matched_rows = matched_rows.drop_duplicates(subset=['Mobile'], keep='first')

In [ ]:
matched_rows.columns

In [ ]:
matched_cols = list(matched_rows.columns.values) #Make the columns to a list

In [ ]:
member_list = list(f"Member_{i+1}" for i in range(64))

In [ ]:
remove_list = ['index', 'house_no', 'age', 'sex', 'pan name', 'ac', 'part_no', 'year', 'state', 'net_electors_male', 'net_electors_female', 'net_electors_third_gender', 'net_electors_total', 'original_or_amendment', 'UGP_Voter']

In [ ]:
print(matched_cols)

In [ ]:
required_cols = [x for x in matched_cols if x not in member_list and x not in remove_list]

In [ ]:
print(required_cols)

In [ ]:
matched_rows.groupby(['UGP', 'WardNo']).nunique() #175 unique wards

In [ ]:
matched_rows.UGP.nunique()

## 8.1 Save matched_rows to csv

#### Open csv files in the relevant folder

In [ ]:
folder = os.path.join(db, pds_input, 'Matched_rows')
all_file_num = []
for file in os.listdir(folder):
    if os.path.isfile(os.path.join(folder, file)):
        try:
            print(file)
            file_num = int(file.split('_')[-1].split('.')[0])
            print(file_num)
            all_file_num.append(file_num)
        except ValueError:
            print(f"{file} does not end with a number.")
            continue
print(all_file_num)

In [ ]:
#Now, find the highest number from the list
max_num = max(all_file_num)
new_file_name = f"matched_rows_{max_num+1}.csv"
print(new_file_name)

In [ ]:
matched_rows.to_csv(os.path.join(db, pds_input, 'Matched_rows', new_file_name), index=False)

In [ ]:
matched_rows = matched_rows[required_cols]

In [ ]:
# matched_rows = matched_rows.sort_values(by=['UGP', 'WardNo', 'RandomOrder'])

In [ ]:
print(f"pds_df_slice:{pds_df_slice.shape[0]}; voter_df: {voter_df.shape[0]}; matched: {matched_rows.shape[0]}")

In [ ]:
cols_to_keep = ['UniqueID', 'RationCard_No', 'UGP', 'WardNo', 'Member_ID', 'Member_Name', 'Father_Name', 'Mobile', 'RandomOrder', 'FPS_Name', 'DistrictName', 'SubDistrictName', 'GramPanchayatName']

In [ ]:
pds_voter_sample = matched_rows[cols_to_keep]

In [ ]:
pds_voter_sample[pds_voter_sample.duplicated(subset='UniqueID', keep=False)]

### Save the new sample

In [ ]:
#Open all files in the sample save directory
folder = os.path.join(db, pds_input, 'Sample')
all_file_num = []
for file in os.listdir(folder):
    if os.path.isfile(os.path.join(folder, file)) and file.endswith('.csv'):
        try:
            print(file)
            file_num = int(file.split('_')[-1].split('.')[0])
            print(file_num)
            all_file_num.append(file_num)
        except ValueError:
            print(f"{file} does not end with a number.")
            continue
print(all_file_num)

In [ ]:
#Now, find the highest number from the list
max_num = max(all_file_num)
new_file_name = f"pds_voter_sample_{max_num+1}.csv"
print(new_file_name)

In [ ]:
#Save the file
pds_voter_sample.to_csv(os.path.join(db, pds_input, 'Sample', new_file_name), index=False)

In [ ]:
pds_voter_sample.shape